IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import seaborn as sns
%matplotlib inline

loading the dataset

In [11]:
data = pd.read_csv("../Dataset/training_data.csv")

In [12]:
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [13]:
data.shape

(31962, 3)

In [14]:
data.size

95886

In [15]:
data.columns

Index(['id', 'label', 'tweet'], dtype='object')

In [16]:
data.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [17]:
data['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

DATA PREPROCESSING

Cleaning the data

In [18]:
# removing handle names
def remove_handle(tweet):
    match = re.findall("@[\w]*",tweet)
    for i in match:
        tweet = re.sub(i,'',tweet)
    return tweet

In [19]:
vector = np.vectorize(remove_handle)

In [20]:
data['tweets without handle'] = vector(data['tweet'])

In [21]:
data.head()

,id,label,tweet,tweets without handle
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can't use cause th...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation


Removing punctuation's,numbers and symbols

In [22]:
data['tweets without handle'] = data['tweets without handle'].str.replace("[^a-zA-Z#]"," ")

In [23]:
data.head()

,id,label,tweet,tweets without handle
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can t use cause th...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now #motivation


In [24]:
data.tail()

,id,label,tweet,tweets without handle
31957,31958,0,ate @user isz that youuu?ðððððð...,ate isz that youuu ...
31958,31959,0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",#sikh #temple vandalised in in #calgary #wso...
31961,31962,0,thank you @user for you follow,thank you for you follow


In [25]:
# REMOVING SHORT WORDS
data['tweets without handle']= data['tweets without handle'].str.lower()
data['tweets without handle'] = data['tweets without handle'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>3]))

In [26]:
#tokenize the words# REMOVING SHORT WORDS
data['tweets without handle']= data['tweets without handle'].str.lower()
data['tweets without handle'] = data['tweets without handle'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>3]))
tokenized_tweets = data['tweets without handle'].apply(lambda x: x.split())
tokenized_tweets.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, #lyft, credit, cause, they, offer, wh...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: tweets without handle, dtype: object

In [27]:
#sTEMMING THE WORDS
from nltk import PorterStemmer
ps = PorterStemmer()
tokenized_tweets = tokenized_tweets.apply(lambda x : [ps.stem(word) for word in x])
tokenized_tweets.head()

0    [when, father, dysfunct, selfish, drag, kid, i...
1    [thank, #lyft, credit, caus, they, offer, whee...
2                              [bihday, your, majesti]
3                     [#model, love, take, with, time]
4                         [factsguid, societi, #motiv]
Name: tweets without handle, dtype: object

In [28]:
#TOKENIZED THE WORD
for i in range(len(tokenized_tweets)):
    tokenized_tweets[i] = ' '.join(tokenized_tweets[i])
data['tweets without handle'] = tokenized_tweets
data.head()

,id,label,tweet,tweets without handle
0,1,0,@user when a father is dysfunctional and is s...,when father dysfunct selfish drag kid into dys...
1,2,0,@user @user thanks for #lyft credit i can't us...,thank #lyft credit caus they offer wheelchair ...
2,3,0,bihday your majesty,bihday your majesti
3,4,0,#model i love u take with u all the time in ...,#model love take with time
4,5,0,factsguide: society now #motivation,factsguid societi #motiv


In [29]:
data.tail()

,id,label,tweet,tweets without handle
31957,31958,0,ate @user isz that youuu?ðððððð...,that youuu
31958,31959,0,to see nina turner on the airwaves trying to...,nina turner airwav tri wrap herself mantl genu...
31959,31960,0,listening to sad songs on a monday morning otw...,listen song monday morn work
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,...",#sikh #templ vandalis #calgari #wso condemn
31961,31962,0,thank you @user for you follow,thank follow


Feature Extraction

Using CountVectorizer

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
vectorizer_bow = CountVectorizer(max_features=6000,stop_words='english',ngram_range=(1,4))
x_bow = vectorizer_bow.fit_transform(data['tweets without handle']).toarray()

In [32]:
x_bow.shape

(31962, 6000)

In [33]:
y_bow = data['label']

In [34]:
y_bow.shape

(31962,)

In [35]:
from sklearn.model_selection import train_test_split
xtrain_bow,xtest_bow,ytrain_bow,ytest_bow = train_test_split(x_bow,y_bow,test_size=0.2,random_state=3)

In [36]:
#checking features names
vectorizer_bow.get_feature_names()

['aap',
 'abandon',
 'abil',
 'abl',
 'abov',
 'abov expect',
 'abrahamhick',
 'abrahamhick lawofattract',
 'abrahamhick lawofattract healthi',
 'abrahamhick lawofattract healthi heal',
 'absolut',
 'abund',
 'abus',
 'academi',
 'accent',
 'accept',
 'accept sale',
 'accept sale photographi',
 'access',
 'accessori',
 'accid',
 'accomplish',
 'accord',
 'account',
 'accus',
 'ace',
 'ach',
 'achiev',
 'acn',
 'acn altwaystoh',
 'acn altwaystoh healthi',
 'acn altwaystoh healthi heal',
 'act',
 'act like',
 'action',
 'activ',
 'activist',
 'actor',
 'actor seeklearn',
 'actor seeklearn stafresh',
 'actorslif',
 'actress',
 'actual',
 'ad',
 'adam',
 'adapt',
 'adapt environ',
 'adapt environ need',
 'adapt environ need tear',
 'add',
 'addict',
 'addit',
 'address',
 'adel',
 'adida',
 'admit',
 'adopt',
 'ador',
 'adult',
 'adv',
 'advanc',
 'advantag',
 'adveis',
 'adventur',
 'advic',
 'advoc',
 'affair',
 'affect',
 'affirm',
 'afford',
 'afraid',
 'africa',
 'african',
 'african 

In [37]:
vectorizer_bow.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 6000,
 'min_df': 1,
 'ngram_range': (1, 4),
 'preprocessor': None,
 'stop_words': 'english',
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [38]:
count_df = pd.DataFrame(x_bow,columns = vectorizer_bow.get_feature_names())

In [39]:
count_df.head()

,aap,abandon,abil,abl,abov,abov expect,abrahamhick,abrahamhick lawofattract,abrahamhick lawofattract healthi,abrahamhick lawofattract healthi heal,...,yum,yummi,yummi cooki pleas,yyc,zara,zealand,zelda,zen,zero,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


By TfidfVectorizer

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tdif = TfidfVectorizer(max_features=6000,stop_words='english',ngram_range=(1,4))

In [41]:
x_tdif = vectorizer_tdif.fit_transform(data['tweets without handle'])

In [42]:
x_tdif = x_tdif.todense()
x_tdif

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
y_tdif=data['label']

In [44]:
y_tdif

0        0
1        0
2        0
3        0
4        0
        ..
31957    0
31958    0
31959    0
31960    1
31961    0
Name: label, Length: 31962, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split
xtrain_tdif,xtest_tdif,ytrain_tdif,ytest_tdif = train_test_split(x_tdif,y_tdif,test_size=0.2,random_state=3)

In [46]:
xtrain_tdif

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
xtrain_tdif.shape

(25569, 6000)

In [48]:
ytrain_tdif.shape

(25569,)

Decision Tree Classifier

By Using Count Vectourizer

In [49]:
from sklearn.tree import DecisionTreeClassifier

In [50]:
dmodel=DecisionTreeClassifier()

In [ ]:
dmodel.fit(xtrain_bow,ytrain_bow)

In [ ]:
pred_y_decision=dmodel.predict(xtest_bow)

In [ ]:
from sklearn import metrics

In [ ]:
accuracy_score=metrics.accuracy_score(ytest_bow,pred_y_decision)

In [ ]:
print(accuracy_score*100)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(classification_report(ytest_bow,pred_y_decision))

In [ ]:
ytest_decision_confusion_matrix=metrics.confusion_matrix(ytest_bow,pred_y_decision)

In [ ]:
print(ytest_decision_confusion_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(ytest_decision_confusion_matrix/np.sum(ytest_decision_confusion_matrix),annot=True,cmap='Blues',fmt='.2f')

By Using Tdidf Vector

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dmodel=DecisionTreeClassifier()

In [ ]:
dmodel.fit(xtrain_tdif,ytrain_tdif)

In [ ]:
pred_y=dmodel.predict(xtest_tdif)

In [ ]:
from sklearn import metrics

In [ ]:
accuracy_score=metrics.accuracy_score(ytest_tdif,pred_y)

In [ ]:
print(accuracy_score*100)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(classification_report(ytest_tdif,pred_y))

In [ ]:
ytest_decision_confusion_matrix=metrics.confusion_matrix(ytest_tdif,pred_y)

In [ ]:
print(ytest_decision_confusion_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(ytest_decision_confusion_matrix/np.sum(ytest_decision_confusion_matrix),annot=True,cmap='Blues',fmt='.2f')